In [2]:
import librosa

In [3]:
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype = "float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

In [5]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=20)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)                             
    return mfccs_scaled_features

In [6]:
# def load_data(test_size=0.2):
#     x,y = [],[]
#     for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
#         label = os.path.basename(file)
#         for audio in glob.glob(file+"\*.wav"):
# #             print(audio)
#             feature = featureExtractor(audio)
#             x.append(feature)
#             y.append(label)
#     return train_test_split(np.array(x), np.array(y), test_size=test_size, random_state=101)

In [7]:
# x_train, x_test, y_train, y_test=load_data()

In [8]:
# print(len(x_train),len(y_train),len(x_test),len(y_test))
# print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [9]:
extracted_features=[]
def load_data():
    c=0
    for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
        label = os.path.basename(file)
        for audio in glob.glob(file+"\*.wav"):
#             print(audio)
            if(label=='hungry'):
                if(c>200):
                    continue
                c+=1
            feature = featureExtractor(audio)
            extracted_features.append([feature,label])

In [10]:
load_data()

In [11]:
# extracted_features

In [12]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-497.25443, 110.52981, -51.0884, -15.25888, 1...",belly_pain
1,"[-497.26553, 151.22801, -56.617985, -1.158033,...",belly_pain
2,"[-421.2681, 153.3781, -55.931305, -7.050015, 6...",belly_pain
3,"[-383.0191, 164.00575, -70.79439, -8.87635, 8....",belly_pain
4,"[-310.78342, 171.10025, -66.29349, 4.647664, 1...",belly_pain


In [13]:
print(extracted_features_df.shape)

(276, 2)


In [14]:
# audio_file_path='belly_pain.wav'
# mfcc=featureExtractor(audio_file_path)
# print(mfcc)
# print(mfcc.shape)

In [15]:
X=np.array(extracted_features_df['feature'].tolist())
Y=np.array(extracted_features_df['class'].tolist())

In [16]:
print(X.shape,Y.shape)
print(Y)

(276, 20) (276,)
['belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'burping' 'burping' 'burping' 'burping' 'burping' 'burping'
 'burping' 'burping' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hung

In [17]:
Y.shape

(276,)

In [18]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(Y))
# Y=np.array(pd.get_dummies(Y))

In [19]:
print(y.shape)
y

(276, 5)


array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [20]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [21]:
X_train.shape

(220, 20)

In [22]:
y_train.shape

(220, 5)

In [23]:
X_test.shape

(56, 20)

In [24]:
y_test.shape

(56, 5)

In [25]:
# BUILDING MODEL
import tensorflow as tf
print(tf.__version__)

2.4.1


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [27]:
num_labels=y.shape[1]

In [28]:
# Dense()

In [29]:
# ANN
model=Sequential()
#first layer
model.add(Dense(100,input_shape=(20,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
          
# final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [31]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [46]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs=50
num_batch_size=32

checkpointer=ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                             verbose=1,save_best_only=True)
start=datetime.now()
model.fit(X_train,y_train,batch_size=num_batch_size,epochs=num_epochs,
          validation_data=(X_test,y_test),callbacks=[checkpointer] )
duration=datetime.now() - start
print("Training completed in time: ",duration)

Epoch 1/50
7/7 [==============================] - 0s 17ms/step - loss: 1.0682 - accuracy: 0.7091 - val_loss: 0.9354 - val_accuracy: 0.7857

Epoch 00001: val_loss improved from inf to 0.93542, saving model to saved_models\audio_classification.hdf5
Epoch 2/50
7/7 [==============================] - 0s 11ms/step - loss: 0.9901 - accuracy: 0.7136 - val_loss: 0.9327 - val_accuracy: 0.7857

Epoch 00002: val_loss improved from 0.93542 to 0.93268, saving model to saved_models\audio_classification.hdf5
Epoch 3/50
7/7 [==============================] - 0s 10ms/step - loss: 1.0165 - accuracy: 0.7045 - val_loss: 0.9330 - val_accuracy: 0.7857

Epoch 00003: val_loss did not improve from 0.93268
Epoch 4/50
7/7 [==============================] - 0s 9ms/step - loss: 1.0735 - accuracy: 0.7000 - val_loss: 0.9225 - val_accuracy: 0.7857

Epoch 00004: val_loss improved from 0.93268 to 0.92251, saving model to saved_models\audio_classification.hdf5
Epoch 5/50
7/7 [==============================] - 0s 10ms/ste

7/7 [==============================] - 0s 9ms/step - loss: 0.9800 - accuracy: 0.7136 - val_loss: 0.9101 - val_accuracy: 0.7857

Epoch 00042: val_loss improved from 0.91603 to 0.91011, saving model to saved_models\audio_classification.hdf5
Epoch 43/50
7/7 [==============================] - 0s 10ms/step - loss: 0.9893 - accuracy: 0.7136 - val_loss: 0.9072 - val_accuracy: 0.7857

Epoch 00043: val_loss improved from 0.91011 to 0.90722, saving model to saved_models\audio_classification.hdf5
Epoch 44/50
7/7 [==============================] - 0s 11ms/step - loss: 1.0017 - accuracy: 0.7091 - val_loss: 0.9079 - val_accuracy: 0.7857

Epoch 00044: val_loss did not improve from 0.90722
Epoch 45/50
7/7 [==============================] - 0s 9ms/step - loss: 1.0296 - accuracy: 0.7136 - val_loss: 0.9123 - val_accuracy: 0.7857

Epoch 00045: val_loss did not improve from 0.90722
Epoch 46/50
7/7 [==============================] - 0s 8ms/step - loss: 0.9872 - accuracy: 0.7091 - val_loss: 0.9127 - val_accu

In [47]:
loss,test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print("loss: ",loss)
print(test_accuracy)

loss:  0.9063510894775391
0.7857142686843872


In [ ]:
filename='burping.wav'
prediction_feature=featureExtractor(filename).reshape(1,-1)
# model.predict_classes(prediction_feature)
predicted_label=model.predict_classes(prediction_feature)
print(predicted_label)
predcited_class=labelencoder.inverse_transform(predicted_label)
predcited_class
# (model.predict(prediction_feature) > 0.5).astype("int32")

In [ ]:
X_test.shape